# BatchWritePolicy Examples

# Import to run %sh magic cell

In [10]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

# Add Java Client POM Dependency

In [2]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client-jdk8</artifactId>
    <version>8.1.1</version>
  </dependency>
</dependencies>

# Add required Imports

In [3]:
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.Record;
import com.aerospike.client.Value;
import com.aerospike.client.policy.RecordExistsAction;
import com.aerospike.client.AerospikeException;
import com.aerospike.client.ResultCode;
import com.aerospike.client.Operation;
System.out.println("Client modules imported.");

Client modules imported.


### Running _asadm_ in iJava
We can run _asadm_ commands inline. Below, we will use the truncation command, which normally requires an interactive confirmation, which we will skip by using the _--no-warn_ flag. No output will be displayed.

In [4]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"

# Generate Test Data

In [5]:
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.BatchPolicy;
import com.aerospike.client.policy.BatchWritePolicy;
import com.aerospike.client.BatchResults;
//import com.aerospike.client.WritePolicy;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.Record;
import com.aerospike.client.Value;
import com.aerospike.client.policy.RecordExistsAction;
import com.aerospike.client.AerospikeException;
//import com.aerospike.client.AerospikeException.BatchRecordArray;
import com.aerospike.client.BatchRecord;
import com.aerospike.client.ResultCode;
import com.aerospike.client.Operation;

import com.aerospike.client.exp.Exp;
import com.aerospike.client.exp.ExpOperation;
import com.aerospike.client.exp.ExpWriteFlags;
import com.aerospike.client.exp.Expression;

import java.util.ArrayList;

System.out.println("Client modules imported.");

AerospikeClient client = new AerospikeClient("127.0.0.1", 3000);

BatchPolicy bP = BatchPolicy.WriteDefault();

bP.totalTimeout = 170;
bP.socketTimeout = bP.totalTimeout;  //Recommended
//bP.socketTimeout = 100;  //Can make smaller than total, but not larger effectively.
//socket < total not recommended for batch writes.

BatchWritePolicy bwP = new BatchWritePolicy();
int iTimeLeft = 6000;
Key[] lKey = new Key[14000];

for(int i=0; i<14000; i++){
  lKey[i] = new Key ("test", "testset", "key"+i);
}
  
Expression tlExp = Exp.build(Exp.val(iTimeLeft));

long timeStart = 0; 
long timeDone = 0;
BatchResults bResults = null;

try {  
       timeStart = System.currentTimeMillis();
       bResults = client.operate(bP, bwP, lKey, 
       ExpOperation.write("timeLeft", tlExp, ExpWriteFlags.DEFAULT)); 
       timeDone = System.currentTimeMillis();
}
catch (AerospikeException.BatchRecordArray ba){ 
    timeDone = System.currentTimeMillis();
    System.out.println("AerospikeException - Message: " + ba.getMessage()
                                        + "\n");
    
    int rc = ba.getResultCode();
    System.out.println("Batch Result Code: "+rc);
    
    BatchRecord[] br = ba.records;
    int numKeys = br.length;
    int TimeoutCount = 0;
    int OKCount = 0;
    int NoResponseCount = 0;
    System.out.println("Num keys: "+numKeys);  
    //These are being divided to each node, going one by one by default
    while (numKeys >0){
        //System.out.println("Key:"+ numKeys +" Key Result Code: "+ br[numKeys-1].resultCode);
        if(br[numKeys-1].resultCode == 9)TimeoutCount++;
        if(br[numKeys-1].resultCode == -15)NoResponseCount++;
        if(br[numKeys-1].resultCode == 0)OKCount++;
        numKeys--;
    }
    System.out.println("Num keys TIMEOUT: "+TimeoutCount);
    System.out.println("Num keys NO_RESPONE: "+NoResponseCount);
    System.out.println("Num keys OK: "+OKCount);
    //Depending on which node completed its set in time, you may all, partial OK, none OK
    //No clue how many actually succeeded.
    //You don't want timeouts!
                        
}
finally {
    if(bResults != null){        
        System.out.println("==> No exception.");
        int numKeys = bResults.records.length;
        System.out.println("Num Keys: "+numKeys);
        int TimeoutCount = 0;
        int OKCount = 0;
        int NoResponseCount = 0;
        while (numKeys >0){
          //System.out.println("Key:"+ numKeys +" Key Result Code: "+ br[numKeys-1].resultCode);
          if(bResults.records[numKeys-1].resultCode == 9)TimeoutCount++;
          if(bResults.records[numKeys-1].resultCode == -15)NoResponseCount++;
          if(bResults.records[numKeys-1].resultCode == 0)OKCount++;
          numKeys--;
        }
        System.out.println("Num keys TIMEOUT: "+TimeoutCount);
        System.out.println("Num keys NO_RESPONE: "+NoResponseCount);
        System.out.println("Num keys OK: "+OKCount);
       
    }
    else{
        System.out.println("==> Exception raised.");
    }
}
//timeDone = System.currentTimeMillis();
System.out.println("Time taken (ms) at client: "+ (timeDone-timeStart));

Client modules imported.
==> No exception.
Num Keys: 14000
Num keys TIMEOUT: 0
Num keys NO_RESPONE: 0
Num keys OK: 14000
Time taken (ms) at client: 218


# Cleanup

In [14]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"